In [43]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [44]:
page = requests.get('https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads')
soup = BeautifulSoup(page.content,'html.parser')

#Find all tables in the page
tables = soup.findAll("table")

In [45]:
team_details = soup.find_all('div', attrs={"class":"toc"})
team_list = []
for i in range(0,32):
    team_name = team_details[0].find_all('li',attrs={"class":"toclevel-2"})[i].contents[0].contents[2].contents[0]
    team_list.append(team_name)

In [46]:
team_df = pd.DataFrame(columns=['Team'])
for i in range(0,len(team_list)):
    team_df.loc[i] = team_list[i]
full_df = pd.DataFrame(columns=['Team Name','Player Name','Club Name','Club Link'])

In [6]:
temp_list=[]
player_list = []
team_list = []
for i in range(0,team_df['Team'].size):
    team_name = team_df['Team'][i];
    players = tables[i].find_all('tr',attrs={"class":"nat-fs-player"})
    for j in range(0,23):
        player_name = players[j].find_all('th')[0].contents[0].contents[0]
        club_name = players[j].find_all('td')[5].contents[2].contents[0]
        club_link = players[j].find_all('td')[5].contents[2].get('href')
        club_link = 'https://en.wikipedia.org' + club_link
        temp_list = [team_name,player_name,club_name,club_link]
        player_list.append(temp_list)
#         print(temp_list)

In [47]:
player_df = pd.DataFrame.from_records(player_list,columns=['Nation','Name','Club','Link'])
club_df = player_df[['Club','Link']]
club_links = club_df.Link.unique()
club_list=[]
temp_list = []
for i in range(0,len(club_links)):
    club_page = requests.get(club_links[i])
    club_soup = BeautifulSoup(club_page.content,'html.parser')
    info_table = club_soup.find('table',attrs={"class":"infobox vcard"})
    info_table_rows = info_table.find_all('tr')
    try:
        for j in range(0,12):
            if(info_table_rows[j].contents[0].contents[0]=='League'):
                league_name=(info_table_rows[j].contents[1].contents[0].get('title'))
    except IndexError:
        league_name = 'Unavailable'
        continue
#     print(club_links[i],'--',league_name)
    temp_list = [club_links[i],league_name]
    club_list.append(temp_list)

https://en.wikipedia.org/wiki/West_Bromwich_Albion_F.C. -- EFL Championship
https://en.wikipedia.org/wiki/Aston_Villa_F.C. -- Premier League
https://en.wikipedia.org/wiki/Los_Angeles_FC -- Major League Soccer
https://en.wikipedia.org/wiki/Wigan_Athletic_F.C. -- EFL Championship
https://en.wikipedia.org/wiki/Zamalek_SC -- Egyptian Premier League
https://en.wikipedia.org/wiki/Liverpool_F.C. -- Premier League
https://en.wikipedia.org/wiki/Stoke_City_F.C. -- EFL Championship
https://en.wikipedia.org/wiki/Arsenal_F.C. -- Premier League
https://en.wikipedia.org/wiki/Kas%C4%B1mpa%C5%9Fa_S.K. -- Süper Lig
https://en.wikipedia.org/wiki/Atromitos_F.C. -- Super League Greece
https://en.wikipedia.org/wiki/PFC_CSKA_Moscow -- Russian Premier League
https://en.wikipedia.org/wiki/FC_Spartak_Moscow -- Russian Premier League
https://en.wikipedia.org/wiki/FC_Akhmat_Grozny -- Russian Premier League
https://en.wikipedia.org/wiki/Villarreal_CF -- La Liga
https://en.wikipedia.org/wiki/FC_Zenit_Saint_Petersbu

https://en.wikipedia.org/wiki/Maccabi_Tel_Aviv_F.C. -- Israeli Premier League
https://en.wikipedia.org/wiki/Red_Star_Belgrade -- Serbian SuperLiga
https://en.wikipedia.org/wiki/Hamburger_SV -- 2. Bundesliga
https://en.wikipedia.org/wiki/Toulouse_FC -- Ligue 1
https://en.wikipedia.org/wiki/1._FC_K%C3%B6ln -- Bundesliga
https://en.wikipedia.org/wiki/Cruz_Azul -- Liga MX
https://en.wikipedia.org/wiki/Club_Atlas -- Liga MX
https://en.wikipedia.org/wiki/C.F._Pachuca -- Liga MX
https://en.wikipedia.org/wiki/Deportivo_Toluca_F.C. -- Liga MX
https://en.wikipedia.org/wiki/Club_Am%C3%A9rica -- Liga MX
https://en.wikipedia.org/wiki/Club_Universidad_Nacional -- Liga MX
https://en.wikipedia.org/wiki/Vissel_Kobe -- J1 League
https://en.wikipedia.org/wiki/Jeonbuk_Hyundai_Motors_FC -- K League 1
https://en.wikipedia.org/wiki/Seongnam_FC -- K League 1
https://en.wikipedia.org/wiki/Ulsan_Hyundai_FC -- K League 1
https://en.wikipedia.org/wiki/Sangju_Sangmu_FC -- K League 1
https://en.wikipedia.org/wiki/S

In [8]:
club_link_df = pd.DataFrame.from_records(club_list,columns=['Link','League'])
player_df = player_df.merge(club_link_df,how='inner',left_on='Link',right_on='Link')
player_df = player_df.drop(['Link'],axis=1)
player_df = player_df.sort_values(by=['Nation','Name','Club'])

In [9]:
player_df.head(5)

,Nation,Name,Club,League
342,Argentina,Cristian Ansaldi,Torino,Serie A
346,Argentina,Federico Fazio,Roma,Serie A
288,Argentina,Gabriel Mercado,Sevilla,La Liga
136,Argentina,Giovani Lo Celso,Paris Saint-Germain,Ligue 1
85,Argentina,Gonzalo Higuaín,Juventus,Serie A


In [10]:
new_df= player_df[player_df.League.isin(['Serie A','La Liga','Premier League','Ligue 1','Eredivisie','Bundesliga'])]

In [18]:
club_series = new_df.groupby(['Club','League']).count().Name

In [19]:
club_df = club_series.to_frame()

In [20]:
club_df.to_csv('..\\Consolidated Data\\World Cup Data at Club Level.csv')

In [ ]:
final_series = player_df.groupby('League').count().Name

In [ ]:
final_df = final_series.to_frame()

In [ ]:
final_df.to_csv('..\\Consolidated Data\\World Cup Data Consolidation.csv')